# 保存和加载模型
  
核心功能：  
torch.save：将序列化对象保存到磁盘。此函数使用Python的pickle模块进行序列化。使用此函数可以保存如模型、tensor、字典等各种对象。  
torch.load：使用pickle的unpickling功能将pickle对象文件反序列化到内存。此功能有助于设备加载数据。  
torch.nn.Module.load_state_dict：使用反序列化函数 state_dict 来加载模型的参数字典。


In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import time 
import os
import copy

#### state_dict 的使用

In [5]:
class ModelClass(nn.Module):
    def __init__(self):
        super(ModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        # [ channels, output, height_2, width_2 ] 
        # channels: 通道数，和上面保持一致，也就是当前层的深度
        # output: 输出的深度
        # height_2: 过滤器filter的高
        # weight_2: 过滤器filter的宽 
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
model = ModelClass()
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)

#### 打印状态字典

In [8]:
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
conv1.weight 	 torch.Size([6, 3, 5, 5])
conv1.bias 	 torch.Size([6])
conv2.weight 	 torch.Size([16, 6, 5, 5])
conv2.bias 	 torch.Size([16])
fc1.weight 	 torch.Size([120, 400])
fc1.bias 	 torch.Size([120])
fc2.weight 	 torch.Size([84, 120])
fc2.bias 	 torch.Size([84])
fc3.weight 	 torch.Size([10, 84])
fc3.bias 	 torch.Size([10])
Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.001, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [1705470830376, 1705470830456, 1705470829736, 1705470829816, 1705470830856, 1705393580456, 1705470829896, 1705470831016, 1705470830296, 1705470831896]}]


#### 保存和加载推理模型

##### 保存/加载state_dict(推荐)
load_state_dict()函数只接受字典对象，而不是保存对象的路径。这就意味着在传给load_state_dict()函数之前，必须反序列化保存的state_dict。例如，无法通过 model.load_state_dict(PATH)来加载模型  
  
##### 保存
```torch.save(model.state_dict(), PATH)```  
##### 加载
```model = ModelClass(*args, **kwargs)    
 model.load_state_dict(torch.load(PATH))    
 model.eval()```  
  
只需要保存模型的参数。在运行推理之前，调用model.eval()去设置dropout和batch normalization层为评估模式，否则可能导致模型推断结果不一致。  
  

In [11]:
torch.save(model, "./model.pth")

In [12]:
model = torch.load("./model.pth")
model.eval()

ModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 保存和加载Checkpoint用于推理 / 训练

In [17]:
epoch = 20

In [18]:
torch.save({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    }, "./models.tar")
#     'loss': loss,

In [22]:
# model = ModelClass(*args, **kwargs)
# optimizer = TheOptimizerClass(*args, **kwargs)

checkpoint = torch.load("./models.tar")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
#loss = checkpoint['loss']

model.eval()
# - or 
# model.train()

ModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 在一个文件中保存多个模型

当保存一个模型由多个`torch.nn.Modules`组成时，例如GAN(对抗生成网络)、sequence-tosequence (序列到序列模型), 或者是多个模型融合, 可以采用与保存常规检查点相同的方法。 换句话说，保存每个模型的 state_dict 的字典和相对应的优化器

In [ ]:
torch.save({ 
    'modelA_state_dict': modelA.state_dict(),   
    'modelB_state_dict': modelB.state_dict(),    
    'optimizerA_state_dict': optimizerA.state_dict(),  
    'optimizerB_state_dict': optimizerB.state_dict(), 
    ...   
}, PATH)

In [ ]:
# modelA = TheModelAClass(*args, **kwargs)
# modelB = TheModelBClass(*args, **kwargs)
# optimizerA = TheOptimizerAClass(*args, **kwargs)
# optimizerB = TheOptimizerBClass(*args, **kwargs)

checkpoint = torch.load(PATH)
modelA.load_state_dict(checkpoint['modelA_state_dict']) 
modelB.load_state_dict(checkpoint['modelB_state_dict'])
optimizerA.load_state_dict(checkpoint['optimizerA_state_dict'])
optimizerB.load_state_dict(checkpoint['optimizerB_state_dict'])

modelA.eval() 
modelB.eval() 
# - or
modelA.train()
modelB.train()

#### 使用在不同模型参数下的热启动模式 
在迁移学习或训练新的复杂模型时，部分加载模型或加载部分模型是常见的情况。
利用训练好的参数，有助于热启动训练过程，并希望帮助你的模型比从头开始训练能够更快地收敛。
无论是从缺少某些键的 state_dict 加载还是从键的数目多于加载模型的 state_dict , 都可以通过在 load_state_dict()函数中将strict参数设置为 **False** 来忽略非匹配键的函数。
如果要将参数从一个层加载到另一个层，但是某些键不匹配，主要修改正在加载的 state_dict 中的 参数键的名称以匹配要在加载到模型中的键即可。


In [ ]:
torch.save(modelA.state_dict(), "./model.pth")

# modelB = TheModelBClass(*args, **kwargs)
modelB.load_state_dict(torch.load(PATH), strict=False)

#### CPU / GPU

In [ ]:
model.load_state_dict(torch.load(PATH, map_location = device)) # 保存到device中

# device = torch.device('cpu') or torch.device("cuda")

model.to(device) # 加载到device中
